<a href="https://colab.research.google.com/github/aTegart/abc-music-gen/blob/main/D3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Getting set up:

You will be connecting to your personal google drive, so make sure you have '/content/drive/My Drive/CS 486 Music Generation/' present (without the ending space, so Celine I changed your folder name). This code now reads from all song folders. Please make sure you are on the latest version of the songs, because on Nov 11 changes were made to filetypes and to remove accented characters. When uploading files to your personal google drive, make sure to have "convert uploads" unchecked in your settings.

Remember you need to authenticate when you remount, and copy the auth code from the link.





In [11]:
from google.colab import drive
#drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


Check position


In [ ]:
!pwd
!cd './drive/My Drive/'
!pwd

/content
/content


## Preprocess the songs

In [12]:
from os import listdir
from os.path import isfile, join
import os

#list songs
mypath = '/content/drive/My Drive/CS 486 Music Generation/' #this was previously "myPath" when it needed to be "mypath" 🤯
#/content/drive/My Drive/CS 486 Music Generation 
#/content/drive/My Drive/CS 486 Music Generation
folders = [f for f in listdir(mypath) if not isfile(join(mypath, f))]
print(folders)
allSongs = []
for folder in folders:
  #if folder == "montreal":
  for song in listdir(mypath + '/' + folder):
    print(folder+" "+song)
    f = open(os.path.join(mypath,folder,song), "r")
    text = f.read()
    allSongs.append(text)
print(len(allSongs))


['montreal', 'tune arch canada - copy', 'canad - copy']
montreal le p_tit bal a l_huile - K D.txt
montreal Turlutte acadienne - K G.txt
montreal Gigue de Salon - K A.txt
montreal The Woolen Cap - K Edor.txt
montreal Reel de la Sauvagine - K A.txt
montreal Violon guerisseur - K D.txt
montreal La valse des jouets - K D.txt
montreal Reel du Verre-Bouteille.txt
montreal Reel Sainte Louise - K D.txt
montreal Valse-clog Lacroix - K D.txt
montreal La Valse Laurencienne - K Bmin.txt
montreal Gigue a Trois - K G.txt
montreal Reel du semeur - K D.txt
montreal Hommage a Yvan Brault - K A.txt
montreal Le Viree dans Villeray - K A.txt
montreal Reel des vieux garcons - K D.txt
montreal The Woodchopper_s Breakdown - K D.txt
montreal reel de vieux moulin - K D.txt
montreal Valse Westphalia - K D.txt
montreal La valse des cerfs-volants - K Bmin.txt
montreal Un air trompeur - K D.txt
montreal Unknown - K D.txt
montreal Le voyage d_Eric St-Pierre - K D.txt
montreal Yves Lambert Irish Melody - K D.txt
mon

In [24]:
print(allSongs[100])

X:49
T:Reel Mimeault
R:reel
M:4/4
L:1/8
K:D
DEFG Addc | BBAF AddA | FDFG Acde | fdeg fdd2 ::
dfed B2BA | Bcdf eAA2 | dfed B2BA | Bcde fdd2 :|



In [14]:
#Combine songs into one long string
spacing = "\n\n"
combined_songs = spacing.join(allSongs)

#Find vocabulary for vectorization (unique characters in string)
uniq_chars = sorted(set(combined_songs))
print(len(uniq_chars))

char_num_map = {c:i for i, c in enumerate(uniq_chars)}
num_char_map = {i:c for i, c in enumerate(uniq_chars)}

#proof that the mapping holds both ways
print('{')
for char,_ in zip(char_num_map, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char_num_map[char]))
print('  ...\n}')

print('{')
for char,_ in zip(num_char_map, range(20)):
    print('  {:4s}: {:3s},'.format(repr(char), num_char_map[char]))
print('  ...\n}')

112
{
  '\t':   0,
  '\n':   1,
  '\x1a':   2,
  ' ' :   3,
  '!' :   4,
  '"' :   5,
  '#' :   6,
  '%' :   7,
  '&' :   8,
  "'" :   9,
  '(' :  10,
  ')' :  11,
  '*' :  12,
  '+' :  13,
  ',' :  14,
  '-' :  15,
  '.' :  16,
  '/' :  17,
  '0' :  18,
  '1' :  19,
  ...
}
{
  0   : 	  ,
  1   : 
  ,
  2   :   ,
  3   :    ,
  4   : !  ,
  5   : "  ,
  6   : #  ,
  7   : %  ,
  8   : &  ,
  9   : '  ,
  10  : (  ,
  11  : )  ,
  12  : *  ,
  13  : +  ,
  14  : ,  ,
  15  : -  ,
  16  : .  ,
  17  : /  ,
  18  : 0  ,
  19  : 1  ,
  ...
}


In [15]:
import numpy as np
#Vectorize combined_songs
# array of a num for each char in the combined_songs string
vsongs = np.array([char_num_map[c] for c in combined_songs])

print ('{} ---- characters mapped to int ----> {}'.format(repr(combined_songs[:10]), vsongs[:10]))
# check that vectorized_songs is a numpy array
assert isinstance(vsongs, np.ndarray), "returned result should be a numpy array"

'X:111\nT:le' ---- characters mapped to int ----> [58 28 19 19 19  1 54 28 77 70]


In [16]:
def batch(vsongs, seqlen, bsize):
  highest_start_index = vsongs.shape[0] - 1 - seqlen
  start_indices = np.random.choice(highest_start_index, bsize)

  input = [vsongs[i : i+seqlen] for i in start_indices]
  output = [vsongs[i+1 : i+1+seqlen] for i in start_indices]

  input = np.reshape(input, [bsize, seqlen])
  output = np.reshape(output, [bsize, seqlen])

  return input, output

testin, testout = batch(vsongs, 5, 2)
#observe how testout is shifted one num forward from testin
print(testin)
print(testout)

[[21 71  3 70 68]
 [93  3 70 20 69]]
[[71  3 70 68  3]
 [ 3 70 20 69 20]]


In [23]:
#more testing, remove later

x_batch, y_batch = batch(vsongs, seqlen=5, bsize=1)

for i, (input_idx, target_idx) in enumerate(zip(np.squeeze(x_batch), np.squeeze(y_batch))):
    print("Step {:3d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(num_char_map[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(num_char_map[target_idx])))

Step   0
  input: 3 (' ')
  expected output: 71 ('f')
Step   1
  input: 71 ('f')
  expected output: 66 ('a')
Step   2
  input: 66 ('a')
  expected output: 72 ('g')
Step   3
  input: 72 ('g')
  expected output: 3 (' ')
Step   4
  input: 3 (' ')
  expected output: 93 ('|')


## Setup Tensorflow and our ML env

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf 

# Check that we are using a GPU, if not switch runtimes
#   using Runtime > Change Runtime Type > GPU
assert len(tf.config.list_physical_devices('GPU')) > 0